In [50]:
import pandas as pd
from bs4 import BeautifulSoup
from glob import glob
import os

In [84]:
with open('valid_files.txt', 'r') as f:
    valid_files = f.read().splitlines()

In [85]:
valid_files

['data\\500118.txt',
 'data\\500163.txt',
 'data\\500226.txt',
 'data\\500390.txt',
 'data\\500393.txt',
 'data\\500424.txt',
 'data\\500443.txt',
 'data\\500563.txt',
 'data\\500584.txt',
 'data\\500600.txt',
 'data\\500699.txt',
 'data\\500757.txt',
 'data\\500763.txt',
 'data\\500794.txt',
 'data\\500846.txt',
 'data\\500925.txt',
 'data\\500940.txt',
 'data\\500970.txt',
 'data\\500972.txt',
 'data\\501000.txt',
 'data\\501027.txt',
 'data\\501057.txt',
 'data\\501091.txt',
 'data\\501107.txt',
 'data\\501174.txt',
 'data\\501178.txt',
 'data\\501218.txt',
 'data\\501222.txt',
 'data\\501341.txt',
 'data\\501459.txt',
 'data\\501460.txt',
 'data\\501488.txt',
 'data\\501543.txt',
 'data\\501579.txt',
 'data\\501607.txt',
 'data\\501631.txt',
 'data\\501677.txt',
 'data\\501680.txt',
 'data\\501694.txt',
 'data\\501709.txt',
 'data\\501738.txt',
 'data\\501746.txt',
 'data\\501789.txt',
 'data\\501878.txt',
 'data\\501892.txt',
 'data\\501916.txt',
 'data\\501919.txt',
 'data\\50205

In [86]:
for indx, file_path in enumerate(valid_files):
    valid_files[indx] = file_path.replace('\\','/')

In [87]:
valid_files

['data/500118.txt',
 'data/500163.txt',
 'data/500226.txt',
 'data/500390.txt',
 'data/500393.txt',
 'data/500424.txt',
 'data/500443.txt',
 'data/500563.txt',
 'data/500584.txt',
 'data/500600.txt',
 'data/500699.txt',
 'data/500757.txt',
 'data/500763.txt',
 'data/500794.txt',
 'data/500846.txt',
 'data/500925.txt',
 'data/500940.txt',
 'data/500970.txt',
 'data/500972.txt',
 'data/501000.txt',
 'data/501027.txt',
 'data/501057.txt',
 'data/501091.txt',
 'data/501107.txt',
 'data/501174.txt',
 'data/501178.txt',
 'data/501218.txt',
 'data/501222.txt',
 'data/501341.txt',
 'data/501459.txt',
 'data/501460.txt',
 'data/501488.txt',
 'data/501543.txt',
 'data/501579.txt',
 'data/501607.txt',
 'data/501631.txt',
 'data/501677.txt',
 'data/501680.txt',
 'data/501694.txt',
 'data/501709.txt',
 'data/501738.txt',
 'data/501746.txt',
 'data/501789.txt',
 'data/501878.txt',
 'data/501892.txt',
 'data/501916.txt',
 'data/501919.txt',
 'data/502059.txt',
 'data/502119.txt',
 'data/502271.txt',


In [91]:
features = ['لون السيارة', 'نوع الوقود', 'أصل السيارة', 'رخصة السيارة', 'نوع الجير', 'الزجاج']
dataset = pd.DataFrame(columns=features)

for file_path in valid_files:
    with open(file_path, encoding='utf-8') as file:
        text = file.read()

    soup = BeautifulSoup(text, 'html.parser')

    list_features_values = []

    for i in soup.find('table', attrs={"class":"list_ads"}).findAll('td'):
        list_features_values.append(i.text)
        
    dict_row_df = dict.fromkeys(features)

    for indx, val in enumerate(list_features_values):
        if val in features:
            dict_row_df[val] = list_features_values[indx+1] 

    dataset.loc[len(dataset)] = dict_row_df


In [92]:
dataset

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني
...,...,...,...,...,...,...
6859,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني
6860,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني
6861,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني
6862,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني


In [98]:
dataset.to_csv('./data_df/dataset_df.csv', index=False)


In [8]:
dataset = pd.read_csv("./data_df/dataset_df.csv")

In [9]:
dataset

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني
...,...,...,...,...,...,...
6859,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني
6860,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني
6861,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني
6862,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني
